In [1]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip.extractall()
zip.close()

--2024-06-21 10:12:44--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.207, 108.177.96.207, 108.177.119.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  41.3MB/s    in 4.5s    

2024-06-21 10:12:49 (35.8 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [4]:
import tensorflow as tf
import os

for dirpath,dirnames,filenames in os.walk("10_food_classes_10_percent"):
  print(f'there are {(dirnames)} directories, {len(filenames)} images in {dirpath}')

there are ['train', 'test'] directories, 0 images in 10_food_classes_10_percent
there are ['fried_rice', 'ice_cream', 'sushi', 'steak', 'pizza', 'grilled_salmon', 'chicken_wings', 'hamburger', 'chicken_curry', 'ramen'] directories, 0 images in 10_food_classes_10_percent/train
there are [] directories, 75 images in 10_food_classes_10_percent/train/fried_rice
there are [] directories, 75 images in 10_food_classes_10_percent/train/ice_cream
there are [] directories, 75 images in 10_food_classes_10_percent/train/sushi
there are [] directories, 75 images in 10_food_classes_10_percent/train/steak
there are [] directories, 75 images in 10_food_classes_10_percent/train/pizza
there are [] directories, 75 images in 10_food_classes_10_percent/train/grilled_salmon
there are [] directories, 75 images in 10_food_classes_10_percent/train/chicken_wings
there are [] directories, 75 images in 10_food_classes_10_percent/train/hamburger
there are [] directories, 75 images in 10_food_classes_10_percent/tra

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"

train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32,
                                               target_size=(224,224),
                                               class_mode='categorical')

test_data = test_datagen.flow_from_directory(test_dir,
                                               batch_size=32,
                                               target_size=(224,224),
                                               class_mode='categorical')

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [29]:
#TensorBoard Callback

from datetime import date

def create_tensorboard_callback(dir_name, exp_name):
  log_dir = dir_name + '/' + exp_name + '/' + date.today().strftime('%Y%m%d')
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir)
  return tensorboard_callback

In [16]:
import tensorflow_hub as hub

resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [19]:
def create_model(model_url, num_classes=10):
  '''Takes a TensorflowHub URL and creates a Keras Sequential model with it.

  Args:
    model_url(str): A TensorflowHub URL (Feature Extractor)
    num_classes(int): Number of output neurons in the output layer

  Returns:
    An uncompiled Keras Sequential model with model_url as feature extractor layer and Dense output layer with num_classes output neurons.
  '''

  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False,
                                           name='feature_extraction_layer',
                                           input_shape=(224,224,3))
  model = tf.keras.Sequential([
      feature_extractor_layer,
      tf.keras.layers.Dense(num_classes, activation='softmax')
  ])

  return model

In [20]:
model_1 = create_model(resnet_url)
model_2 = create_model(efficientnet_url)


In [31]:
model_1.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

model_2.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [32]:
model_1.fit(train_data,
            epochs=5,
            steps_per_epoch=len(train_data),
            validation_data=test_data,
            validation_steps=len(test_data),
            callbacks=create_tensorboard_callback(dir_name='thub',
                                                  exp_name='RESNET50V2'))

Epoch 1/5
24/24 [==============================] - 14s 521ms/step - loss: 0.2015 - accuracy: 0.9693 - val_loss: 0.6347 - val_accuracy: 0.7968
Epoch 2/5
24/24 [==============================] - 12s 532ms/step - loss: 0.1332 - accuracy: 0.9893 - val_loss: 0.6227 - val_accuracy: 0.7992
Epoch 3/5
24/24 [==============================] - 12s 503ms/step - loss: 0.1025 - accuracy: 0.9947 - val_loss: 0.6143 - val_accuracy: 0.7984
Epoch 4/5
24/24 [==============================] - 11s 462ms/step - loss: 0.0817 - accuracy: 0.9960 - val_loss: 0.6230 - val_accuracy: 0.8000
Epoch 5/5
24/24 [==============================] - 12s 491ms/step - loss: 0.0659 - accuracy: 0.9960 - val_loss: 0.6265 - val_accuracy: 0.8004


In [33]:
model_2.fit(train_data,
            epochs=5,
            steps_per_epoch=len(train_data),
            validation_data=test_data,
            validation_steps=len(test_data),
            callbacks=create_tensorboard_callback(dir_name='thub',
                                                  exp_name='EFFICIENTNET80'))

Epoch 1/5
24/24 [==============================] - 24s 587ms/step - loss: 1.7967 - accuracy: 0.4733 - val_loss: 1.2540 - val_accuracy: 0.7472
Epoch 2/5
24/24 [==============================] - 13s 544ms/step - loss: 1.0228 - accuracy: 0.7907 - val_loss: 0.8443 - val_accuracy: 0.8240
Epoch 3/5
24/24 [==============================] - 10s 436ms/step - loss: 0.7298 - accuracy: 0.8413 - val_loss: 0.6861 - val_accuracy: 0.8460
Epoch 4/5
24/24 [==============================] - 10s 433ms/step - loss: 0.5912 - accuracy: 0.8733 - val_loss: 0.6048 - val_accuracy: 0.8528
Epoch 5/5
24/24 [==============================] - 12s 534ms/step - loss: 0.5025 - accuracy: 0.8840 - val_loss: 0.5516 - val_accuracy: 0.8628
